# Loading Google 8M




In [1]:
import tensorflow as tf 
from youtube_transcript_api import YouTubeTranscriptApi


2022-09-30 01:03:40.796220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# glob all .tfrecord files in ./YT_8M_data directory
filenames = tf.io.gfile.glob('./YT_8M_data/*.tfrecord')

In [3]:
import requests
def fake_to_real_ytid(ytid):
  assert len(ytid) == 4
  x = f"http://data.yt8m.org/2/j/i/{ytid[0:2]}/{ytid}.js"
  r = requests.get(x)
  if r.status_code != 200:
    raise Exception("Video was probably removed:", r.text)
    
  return r.text[10:21]

In [5]:
def get_captions(captions, start):
  end = start + 5
  relevant_captions = []
  for caption in captions:
    caption_start = caption['start']
    caption_end = caption_start + caption['duration']
    # Case 1: started before
    if (end > caption_start and end < caption_end) or (start > caption_start and start < caption_end):
      relevant_captions.append(caption['text'])
  return relevant_captions

In [9]:
# save python dict to json file named 'captions.json'
import json
def save_captions(captions):
  with open('./8M_all_captions.json', 'w') as f:
    json.dump(all_captions, f)

In [ ]:
# todo use ray for parallel downloading

TEST_FILE_LIMIT = 2000

# DATA FORMAT: captions_dict[video_id] = [caption1, caption2, ...]
all_captions = {}
num_no_caption = 0

for itr, file in enumerate(filenames[:TEST_FILE_LIMIT]):
  raw_dataset = tf.data.TFRecordDataset(file)
  # for raw_record in raw_dataset:
  for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    start_times = example.features.feature['segment_start_times'].int64_list.value
    fake_id = example.features.feature['id'].bytes_list.value[0].decode("utf-8")

    try:
      yt_id = fake_to_real_ytid(fake_id)
    except Exception as e:
      # probably 403 error, forbidden (removed video)
      num_no_caption += 1
      continue

    try:
      url = YouTubeTranscriptApi.get_transcript(yt_id)
    except Exception as e:
      num_no_caption += 1
      # print("No captions for", yt_id)#, e) # probably no captions
      continue

    caption_list = []
    # daniel put your code here
    for index, start_time in enumerate(start_times):
      captions = get_captions(url, start_time)
      concat_captions = " ".join(captions)
      caption_list.append(concat_captions)
      # todo: Add this caption_dict to the current dataframe 
    # print(caption_dict)
    all_captions[yt_id] = caption_list
    if itr % 10 == 0: print(f"Iteration {itr}: running missing_caption ratio (higher is bad): ", num_no_caption / itr)
    break

print("No caption ratio: ", num_no_caption / TEST_FILE_LIMIT)
print(all_captions)

save_captions_to_json(all_captions)

In [15]:
save_captions_to_json(all_captions)

In [14]:
# save python dict to json file named 'captions.json'
import json
def save_captions_to_json(captions):
  with open('8M_all_captions.json', 'w') as f:
    json.dump(all_captions, f)

In [ ]:
len(all_captions)

In [ ]:
saved_all_captions = all_captions
saved_all_captions

In [ ]:
tf.data.TFRecordDataset(file)

In [ ]:
import tensorflow as tf 
raw_dataset = tf.data.TFRecordDataset("./YT_8M_data/validate0002.tfrecord")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)
    # print(example.features.feature.keys())
    # print(example.features.feature['labels'].int64_list.value)

id = example.features.feature['id'].bytes_list.value[0].decode("utf-8")
# labels = example.features.feature['labels'].int64_list.value
# segment_labels = example.features.feature['segment_labels'].int64_list.value
# segment_scores = example.features.feature['segment_scores'].float_list.value
# segment_start_times = example.features.feature['segment_start_times'].float_list.value
# segment_end_times = example.features.feature['segment_end_times'].float_list.value
print(id)

# getting real YT vid IDs
ID conversion: https://research.google.com/youtube8m/video_id_conversion.html

construct a URI like /AB/ABCD.js (note: first 2 characters are repeated!), and append it to the URL data.yt8m.org/2/j/i.


data.yt8m.org/2/j/i/nX/nXSc.js

In [ ]:
def fake_to_real_ytid(ytid):
  assert len(ytid) == 4
  return f"data.yt8m.org/2/j/i/{ytid[0:2]}/{ytid}.js"

In [ ]:
penis = fake_to_real_ytid('pFac')
penis

In [ ]:
!wget data.yt8m.org/2/j/i/pF/pFac.js

# Fetching transcripts from youtube


In [ ]:
'''
Full example. The benefit is we get the language code, and we can translate it if necessary.
More programming work to manually check for translation. 
'''

# video_id = ['bAYa_YjdHvI', 'AjpzS1xogJc']
video_id = 'V1Z3OhDHzB4'

from youtube_transcript_api import YouTubeTranscriptApi

# from Exception import suppress 
from contextlib import suppress

# retrieve the available transcripts
with suppress(Exception):
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    # print("")
else: 

    
# iterate over all available transcripts
for transcript in transcript_list:

    # the Transcript object provides metadata properties
    print(
        transcript.video_id,
        transcript.language_code,
        transcript.language,
        # whether it has been manually created or generated by YouTube
        # transcript.is_generated,
        # whether this transcript can be translated or not
        "is_translatable", transcript.is_translatable,
        # a list of languages the transcript can be translated to
        # transcript.translation_languages,
    )

    # fetch the actual transcript data
    print(transcript.fetch())

    # translating the transcript will return another transcript object
    # print(transcript.translate('en').fetch())

# you can also directly filter for the language you are looking for, using the transcript list
# transcript = transcript_list.find_transcript(['en'])  

# or just filter for manually created transcripts  
# transcript = transcript_list.find_manually_created_transcript(['en'])  

# or automatically generated ones  
# transcript = transcript_list.find_generated_transcript(['en'])

In [ ]:
transcript = transcript_list.find_transcript(['en'])  
print(transcript.fetch())

In [ ]:
# GET TRANSCRIPTS IN JSON. 

from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import JSONFormatter

# Must be a single transcript.
transcript = YouTubeTranscriptApi.get_transcript('bAYa_YjdHvI')

formatter = JSONFormatter()

# .format_transcript(transcript) turns the transcript into a JSON string.
json_formatted = formatter.format_transcript(transcript)

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
video_id = 'bAYa_YjdHvI'
# transcript_list = YouTubeTranscriptApi.list_transcripts(ytid)

transcript_list = YouTubeTranscriptApi.get_transcripts([video_id], languages=['en'])

# transcript.language_code
# print(transcript_list)
transcript_list[0].language_code
# print(transcript_list.find_transcript(['en']))


In [ ]:
video_id = 'bAYa_YjdHvI'

from youtube_transcript_api import YouTubeTranscriptApi
  
# assigning srt variable with the list
# of dictonaries obtained by the get_transcript() function
srt = YouTubeTranscriptApi.get_transcript(video_id)
  
# prints the result
print(srt)

# learning to raise custom exceptions

In [ ]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class empty_string_error(Error):
    """Raised when the input value is too large"""
    # return "String is empty, dumbass"
    pass

try:
  if("Your Condition"):
    raise empty_string_error
  else:
    print("OUTPUT")
except empty_string_error as e:
  print("raising empty string error", e)
  print(" ")
finally:
  print("Mandatory code")